In [1]:
import keras
import numpy as np
from  keras.utils import np_utils
import keras.utils as utils
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.datasets import cifar10
from keras.models import Model
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from scipy import misc
import scipy

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Bottleneck features can be downloaded from https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5834b432_vgg-100/vgg-100.zip . I you have python 2 installed, unpickle them like this(remeber using Python 3)

import pickle
with open("vgg_cifar10_bottleneck_features_validation.p", "rb") as f:
    w = pickle.load(f)
pickle.dump(w, open("cifarval.pkl","wb"), protocol=2)
with open("vgg_cifar10_100_bottleneck_features_train.p", "rb") as f:
    v = pickle.load(f)

pickle.dump(v, open("cifartrain.pkl","wb"), protocol=2)


### Loading bottkeneck features

In [37]:
import pickle
def load_bottleneck(trainfile,valfile):
    f = open(trainfile, 'r')   # 'r' for reading; can be omitted
    mydicttrain = pickle.load(f)         # load file content as mydict
    f.close()
    g=open(valfile,'r')
    mydictval=pickle.load(g)
    g.close
    x_train=mydicttrain['features']
    y_train=mydicttrain['labels']
    x_val=mydictval['features']
    y_val=mydictval['labels']
    return x_val,y_val,x_train,y_train


In [41]:
x_train,y_train,x_val,y_val=load_bottleneck('cifartrain.pkl','cifarval.pkl')
print "x_train and x_val shape are respectively "+str(x_train.shape)+"and "+str(x_val.shape)
print "y_train and y_val shape are respectively "+str(y_train.shape)+"and "+str(y_val.shape)

x_train and x_val shape are respectively (10000, 1, 1, 512)and (1000, 1, 1, 512)
y_train and y_val shape are respectively (10000, 1)and (1000, 1)


In [48]:
num_classes=len(np.unique(y_train))
print "Unique classes are "+ str(num_classes)

Unique classes are 10


In [65]:
from keras.callbacks import ModelCheckpoint
input_shape = x_train.shape[1:]
inp = Input(shape=input_shape)
x = Flatten()(inp)
x = Dense(num_classes, activation='softmax')(x)
model = Model(inp, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    # train model
checkpointer = ModelCheckpoint(filepath='modelwithtransfer.best.hdf5', 
                               verbose=1, save_best_only=True)
# model.fit(features, y_train, batch_size=50, epochs=50,
#           validation_split=0.2, callbacks=[checkpointer],
#           verbose=2, shuffle=True)
model.fit(x_train,y_train,batch_size=50,epochs=50,validation_split=0.2,callbacks=[checkpointer],verbose=2,shuffle=True)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
 - 1s - loss: 1.6432 - acc: 0.5199 - val_loss: 0.9230 - val_acc: 0.7010

Epoch 00001: val_loss improved from inf to 0.92304, saving model to modelwithtransfer.best.hdf5
Epoch 2/50
 - 1s - loss: 0.7382 - acc: 0.7525 - val_loss: 0.7277 - val_acc: 0.7625

Epoch 00002: val_loss improved from 0.92304 to 0.72773, saving model to modelwithtransfer.best.hdf5
Epoch 3/50
 - 1s - loss: 0.5791 - acc: 0.8039 - val_loss: 0.6270 - val_acc: 0.7915

Epoch 00003: val_loss improved from 0.72773 to 0.62701, saving model to modelwithtransfer.best.hdf5
Epoch 4/50
 - 1s - loss: 0.4964 - acc: 0.8345 - val_loss: 0.5919 - val_acc: 0.8020

Epoch 00004: val_loss improved from 0.62701 to 0.59185, saving model to modelwithtransfer.best.hdf5
Epoch 5/50
 - 1s - loss: 0.4414 - acc: 0.8506 - val_loss: 0.5694 - val_acc: 0.8085

Epoch 00005: val_loss improved from 0.59185 to 0.56944, saving model to modelwithtransfer.best.hdf5
Epoch 6/50
 - 1s - loss: 0.4006 - ac

In [67]:
model.load_weights('modelwithtransfer.best.hdf5')

# evaluate test accuracy
score = model.evaluate(x_val, y_val, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 84.6000%
